In [1]:
! python -m pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 40.3 MB/s eta 0:00:00


In [2]:
from gurobipy import Model, GRB, quicksum

# Insiemi
I = ["aranciata", "concentrato"]  # prodotti
J = ["arance", "acqua"]           # materie prime

# Parametri

r = {
    "aranciata": 0.8,   # 80 cent a litro
    "concentrato": 2.0  # 2 euro a kg
}


b = {
    "arance": 1000,   # kg di arance
    "acqua": 10000,   # litri di acqua
}

A = {
    ("aranciata", "arance"): 0.5,   # 0.5 kg di arance per 1 litro di aranciata
    ("aranciata", "acqua"):  2.0,   # 2 litri di acqua consumati

    ("concentrato", "arance"): 5.0,  # 5 kg di arance per 1 kg di concentrato
    ("concentrato", "acqua"):  -0.5, # 0.5 litri di acqua "prodotti"
}

# Modello
model = Model("Aranciata_Concentrato")

## Variabili decisionali
x = model.addVars(I, vtype=GRB.INTEGER, lb=0, name="x")

## Vincoli

### (1)
for j in J:
    model.addConstr(
        quicksum(A[(i, j)] * x[i] for i in I) <= b[j],
        name=f"risorsa_{j}"
    )

# Funzione obiettivo: massimizzare il ricavo totale
model.setObjective(quicksum(r[i] * x[i] for i in I), GRB.MAXIMIZE)

# Risolvi il modello
model.optimize()

# Stampa soluzione
if model.status == GRB.OPTIMAL:
    print(f"Ricavo totale: {model.objVal:.2f} euro")
    print("Piano di produzione:")
    for i in I:
        qty = x[i].X
        if qty > 0.5:  # soglia per evitare di stampare quasi-zeri numerici
            print(f"  {i}: {qty:.0f} confezioni")
else:
    print("Nessuna soluzione ottima trovata. Stato del modello:", model.status)

Restricted license - for non-production use only - expires 2027-11-29
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros (Max)
Model fingerprint: 0x61f1803e
Model has 2 linear objective coefficients
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-01, 5e+00]
  Objective range  [8e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+04]
Found heuristic solution: objective 1600.0000000
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 1600 

Optimal solution found (tolerance 1.00e-04)
Best objecti